1. Preprocessing the Data
Before training your model, you need to preprocess your text data to convert it into a format that the machine learning algorithm can work with.

Tokenization: Split the text into words or tokens.
Removing Noise: Filter out unnecessary data, like special characters, numbers, and punctuation.
Stopwords Removal: Remove common words that might not contribute much to the sentiment analysis.
Stemming or Lemmatization: Reduce words to their base or root form.
Python's nltk library can be very helpful for these tasks.

2. Vectorization
After preprocessing, convert the text data into numerical features. Common techniques include:

Bag of Words: Count the frequency of each word in each document.
TF-IDF (Term Frequency-Inverse Document Frequency): Weigh the frequencies by how unique they are to the document in the context of the entire dataset.
You can use CountVectorizer or TfidfVectorizer from Python's sklearn.feature_extraction.text for this step.

3. Training the Naïve Bayes Classifier
With the data preprocessed and vectorized, you can now train the Naïve Bayes classifier.

Choosing the Model: Depending on the nature of your feature vectors, choose between different types of Naïve Bayes classifiers available in sklearn.naive_bayes. For instance, MultinomialNB works well with counts (like from Bag of Words), while GaussianNB is used when features are continuous.
Training the Model: Use the training dataset to train the model.
4. Model Evaluation
Evaluate the model using your validation dataset to see how well it performs.

Accuracy: Measure how often the classifier correctly predicts the sentiment.
Confusion Matrix: Useful to see the true positives, true negatives, false positives, and false negatives.
Precision, Recall, and F1-Score: These metrics will help you understand the balance between precision and recall, giving you a better picture of overall performance.
5. Fine-tuning and Improvements
Depending on the initial results, you may need to adjust your preprocessing, tweak the model, or choose different parameters.

Hyperparameter Tuning: Adjust parameters like alpha in Naïve Bayes.
Cross-Validation: Use cross-validation to ensure your model's performance is stable across different subsets of your data.
6. Implementation
Implement the model to make predictions on new tweets to see the sentiment.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Downloads
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\matt7\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\matt7\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\matt7\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# Load data
data = pd.read_csv('twitter_training.csv')

In [3]:
data.head()

,Tweet ID,Entity,Sentiment,Tweet Content
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [4]:
# 1. Preprocessing 

In [6]:
def preprocess_text(text):
    # Check nan values
    if pd.isna(text):
        return ""

    # Tokenization & lowercasing
    tokens = word_tokenize(text.lower())
    
    # Removing punctuation & nums
    words = [word for word in tokens if word.isalpha()]
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if not word in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    lemmatized = [lemmatizer.lemmatize(word) for word in words]
    
    # Join words 
    return ' '.join(lemmatized)

# Apply
data['Processed Tweet Content'] = data['Tweet Content'].apply(preprocess_text)


In [7]:
data.head()

,Tweet ID,Entity,Sentiment,Tweet Content,Processed Tweet Content
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...,im getting borderland murder
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...,coming border kill
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...,im getting borderland kill
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...,im coming borderland murder
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...,im getting borderland murder


In [15]:
# 2. Vectorization

In [8]:
# TF-IDF Vectorizer
vectorizer = TfidfVectorizer()

# Fit the vectorizer & transform 
tfidf_matrix = vectorizer.fit_transform(data['Processed Tweet Content'])

In [9]:
# Get feature names 
feature_names = vectorizer.get_feature_names_out()

# Preview feature names
print(feature_names[:25]) 

['aa' 'aaa' 'aaaaaaaaaaaa' 'aaaaaaaaaaaaa'
 'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa'
 'aaaaaaaaaaages' 'aaaaaaaaaages' 'aaaaaasee' 'aaaahhh' 'aadii' 'aadil'
 'aajtak' 'aall' 'aamaavpjyc' 'aarogya' 'aaron' 'aaroncarter'
 'aarongreenberg' 'aat' 'aatmanirbhar' 'aatmanirvar' 'ab' 'aback' 'abah'
 'abandon']


In [10]:
# 3. Training the Naïve Bayes Classifier

In [11]:
# Prepare your features (TF-IDF vectors) & labels (Sentiments)
X = tfidf_matrix
y = data['Sentiment']

# Split data training and test 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
# Initialize the Multinomial Naïve Bayes classifier
model = MultinomialNB()

# Train 
model.fit(X_train, y_train)

MultinomialNB()

In [13]:
# Predict sentiments test set
y_pred = model.predict(X_test)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Classification report
print(classification_report(y_test, y_pred))

Accuracy: 0.7036218785566044
              precision    recall  f1-score   support

  Irrelevant       0.93      0.41      0.57      2592
    Negative       0.64      0.88      0.74      4519
     Neutral       0.78      0.59      0.67      3596
    Positive       0.69      0.79      0.74      4230

    accuracy                           0.70     14937
   macro avg       0.76      0.67      0.68     14937
weighted avg       0.74      0.70      0.69     14937



Analyzing Model Evaluation Results
Accuracy: The model correctly predicted the sentiment of tweets about 70.36% of the time across all classes, which is fair for a preliminary model but suggests there is room for improvement.

Precision and Recall:

Irrelevant: High precision (0.93) but low recall (0.41) suggests that while most of the predictions for this class are correct, the model is missing a significant number of 'Irrelevant' tweets.
Negative: Decent precision (0.64) with high recall (0.88). The model is capable of identifying most 'Negative' tweets, though it also includes a moderate number of false positives.
Neutral: Both precision (0.78) and recall (0.59) are moderate, indicating a balanced but not highly accurate performance on this class.
Positive: Moderate precision (0.69) and good recall (0.79). The model is relatively successful in identifying 'Positive' tweets but with some false positives.
F1-Score: Gives a balance between precision and recall. We notice that 'Negative' and 'Positive' sentiments have the highest F1 scores, which are the classes better handled by the model compared to 'Irrelevant' and 'Neutral'.

In [14]:
# 5. Fine-tuning and Improvements

In [15]:
# Adjust parameters
vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, ngram_range=(1, 2))

# Fit & transform 
tfidf_matrix = vectorizer.fit_transform(data['Processed Tweet Content'])

# Split data
X = tfidf_matrix
y = data['Sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [16]:
# Initialize the Multinomial Naïve Bayes classifier
model = MultinomialNB(alpha=0.1) 

# Train model
model.fit(X_train, y_train)

# Predict sentiments test 
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

print(classification_report(y_test, y_pred))


Accuracy: 0.911093258351744
              precision    recall  f1-score   support

  Irrelevant       0.96      0.86      0.91      2592
    Negative       0.87      0.95      0.91      4519
     Neutral       0.93      0.89      0.91      3596
    Positive       0.91      0.91      0.91      4230

    accuracy                           0.91     14937
   macro avg       0.92      0.91      0.91     14937
weighted avg       0.91      0.91      0.91     14937



In [17]:
# 6. Implementation

In [18]:
# Step 1 Preprocessing

# Load validation dataset
validation_data = pd.read_csv('twitter_validation.csv')

validation_data['Processed Tweet Content'] = validation_data['Tweet Content'].apply(preprocess_text)

In [19]:
# Step 2: Vectorize the Preprocessed Text
X_validation = vectorizer.transform(validation_data['Processed Tweet Content'])

In [20]:
# Step 3: Make Predictions
y_validation_pred = model.predict(X_validation)

In [21]:
# Step 4: Evaluate the Predictions

# True
y_validation_true = validation_data['Sentiment']

# Accuracy
accuracy_validation = accuracy_score(y_validation_true, y_validation_pred)
print(f"Validation Accuracy: {accuracy_validation}")

# Classification report
print(classification_report(y_validation_true, y_validation_pred))

Validation Accuracy: 0.981
              precision    recall  f1-score   support

  Irrelevant       0.99      0.98      0.99       172
    Negative       0.97      0.98      0.98       266
     Neutral       0.99      0.97      0.98       285
    Positive       0.97      0.99      0.98       277

    accuracy                           0.98      1000
   macro avg       0.98      0.98      0.98      1000
weighted avg       0.98      0.98      0.98      1000



These results look fantastic! Your model achieved a high validation accuracy of approximately 98.1%, indicating its strong performance on the validation dataset. Here's a breakdown of the evaluation metrics:

Precision, Recall, and F1-Score: Across all sentiment classes ('Irrelevant', 'Negative', 'Neutral', and 'Positive'), precision, recall, and F1-score values are consistently high, ranging from approximately 97% to 99%. This indicates that the model is performing exceptionally well in correctly identifying tweets belonging to each sentiment category, with balanced precision and recall.

Macro and Weighted Averages: Both macro and weighted averages for precision, recall, and F1-score are approximately 0.98, which confirms the overall effectiveness of the model across all sentiment classes.

Conclusion
With a validation accuracy of around 98.1%, your model demonstrates remarkable performance in accurately predicting the sentiment of tweets in the validation dataset. This high accuracy indicates that your model generalizes well to unseen data, making it suitable for real-world applications.

If you're satisfied with these results, you can consider this step successful. However, it's always a good practice to continue monitoring and evaluating the model's performance over time, especially as new data becomes available.